# Dolev-Strong in Python 

This will be a self-contained notebook which implements the Dolev-Strong Consensus protocol in python. If you want to understand how it works checkout the resources below (specifically the first one!)

<div align="center">
<img src="2022-10-15-12-20-04.png" width="450" height="300">
</div>

### Resources 

http://elaineshi.com/docs/blockchain-book.pdf - best resource imo on basics

https://decentralizedthoughts.github.io/ - second best; good for cross-checking

https://www.youtube.com/watch?v=rKGhbC6Uync - good video series to learn more from 

https://nakamotoinstitute.org/literature/ - interesting historical notes on btc

### Signature Chains
```python
# signature chain outline:
# > SIGN
# 1 sig: 
    # sig = sign(msg)
# 2 sig: 
    # sig2 = sign(sig)
# > VERIFY
# verify(sig2) == sig
# verify(sig) == msg
```

In [1]:
import rsa 
from dataclasses import dataclass

@dataclass
class SignatureItem: 
    msg: bytes
    sig: bytes 
    pk: bytes 

SignatureChain = list[SignatureItem]

def verify(msg, sig, pk) -> bool:
    try: 
        rsa.verify(msg, sig, pk)
        return True
    except rsa.VerificationError:
        return False

In [32]:
users = [rsa.newkeys(512) for _ in range(2)]
pks, sks = list(zip(*users))

# sign 
msg = b'0x19'

sig_chain: SignatureChain = []
for i, sk in enumerate(sks):
    sig = rsa.sign(msg, sk, 'SHA-1')

    # if i == 1: # (verification fails)
    #     # random sig
    #     sig = b'\x13\x84"m\x16\xaf\xdeuw\xbf\x02\x86Nl\xe2\x17\xe6\xfc\xe4:\xb4\x04\xacW\x06\x8d^%\xc5\xe5%<\r)"\x0e\x8d33\xc1+\x83ZE\r\xbdHO\x93C\xbf\xca\xaa\x00\xb7\x18[\xf7#\x94\xc7\x98y\x14'

    sig_item = SignatureItem(msg, sig, pks[i])
    sig_chain.append(sig_item)
    msg = sig

# verify 
sig_item: SignatureItem
for sig_item in sig_chain[::-1]:
    # if fails = throws error
    assert verify(sig_item.msg, sig_item.sig, sig_item.pk), 'verification failed..'

### Dolev Nodes

In [31]:
class Node: 
    def __init__(self, name) -> None:
        self.pk, self.sk = rsa.newkeys(512)
        self.name = name 
        self.chain = []
        self.network = []
    
    def set_network(self, network: list):
        _network = []
        for n in network: 
            if n.name != self.name: _network.append(n)
        self.network = _network

    def recv(self, msg: bytes, sig_chain: SignatureChain = []):
        assert len(sig_chain) == 0 or msg == sig_chain[0].msg

        # verify signature chain so far
        sig_item: SignatureItem
        for sig_item in sig_chain[::-1]:
            # if fails = throws error
            result = verify(sig_item.msg, sig_item.sig, sig_item.pk)
            if not result: 
                print('sig verification failed...')
                return 

        # check if full network signed -- if so, add it to the chain! 
        already_signed = any([item.pk == self.pk for item in sig_chain])
        network_signed = len(sig_chain) == len(self.network) + 1
        is_last_sig = len(sig_chain) == len(self.network) and not already_signed
        if network_signed or is_last_sig:
            print(f'node {self.name}: msg {msg} fully verified...')
            self.chain.append(msg)

        if not already_signed: 
            # add this node's signature
            last_sig = msg if len(sig_chain) == 0 else sig_chain[-1].sig
            sig = rsa.sign(last_sig, self.sk, 'SHA-1')
            sig_item = SignatureItem(last_sig, sig, self.pk)
            sig_chain.append(sig_item)

            # broadcast signature to other nodes
            for node in self.network:
                node.recv(msg, sig_chain)


In [32]:
node1 = Node(1)
node2 = Node(2)

network = [node1, node2]
for n in network:
    n.set_network(network)

# user sends msg to nodes
msg = b'0x19'
network[0].recv(msg)

# check the chain 
for n in network:
    print(f'node {n.name} chain: {n.chain}')

node 2: msg b'0x19' fully verified...
node 1: msg b'0x19' fully verified...
node 1 chain: [b'0x19']
node 2 chain: [b'0x19']
